In [1]:
from base import *
from utils import pauli_sum, entropy
import plot_utils
from plot_tools import *
import qiskit as qk
from qiskit import QuantumCircuit, Aer, execute
from qiskit.primitives import Estimator
# from qiskit.utils import QuantumInstance
# from qiskit.algorithms import VQE
from qiskit.algorithms.minimum_eigensolvers import VQE
from qiskit.algorithms.optimizers import SLSQP, ADAM, COBYLA
from qiskit.opflow import X, Z, I, Y, AerPauliExpectation, PauliSumOp
from qiskit.circuit import Parameter 
from qiskit.circuit.library import TwoLocal
import numpy as np

In [2]:
from algorithms import Vqe 

## J=2

In [3]:
# constants
eps = 1
z_coeff = 0.5 * eps



In [31]:

def H(V,W):
    return np.array([[-2*eps, 0, np.sqrt(6)*V, 0, 0],
                  [0, -eps+3*W, 0, 3*V, 0],
                  [np.sqrt(6)*V, 0, 4*W, 0, np.sqrt(6)*V],
                  [0, 3*V, 0, eps+3*W, 0],
                  [0, 0, np.sqrt(6)*V, 0, 2*eps]])


# def my_H(v):
    
#     x_coeff = -v/2
#     y_coeff = v/2
#     return pauli_sum([('ZIII', z_coeff), ('IZII', z_coeff), ('IIZI', z_coeff), ('IIIZ', z_coeff), # all 1 qubit combinations
#                       ('XXII', x_coeff), ('XIXI', x_coeff), ('XIIX', x_coeff), ('IXXI', x_coeff), ('IXIX', x_coeff), ('IIXX', x_coeff), # all 2 qubit combinations
#                       ('YYII', y_coeff), ('YIYI', y_coeff), ('YIIY', y_coeff), ('IYYI', y_coeff), ('IYIY', y_coeff), ('IIYY', y_coeff)])

# my_H(0)

In [34]:
vs = np.arange(0, 1, 0.1)
ws = np.arange(0, 1, 0.1)
# vs, ws = np.meshgrid(vs, ws) 
print(vs.shape, ws.shape)
eigs = np.zeros((5, len(vs), len(ws)))

for i, v in enumerate(vs):
    for j, w in enumerate(ws):
        # print(H(v,w))
        eigvals, eigvecs = np.linalg.eig(H(v, w))
        # print(eigvals)
        eigs[:, i, j] = sorted(eigvals)


t = np.arange(0,10)
t = t/10
sns.heatmap(eigs[0].T, cmap='viridis', xticklabels=t, yticklabels=t)

# Set labels and title
plt.xlabel('v')
plt.ylabel('w')
# plt.show()
plt.title('Eigenvalues Heatmap')
# eigs[0, 2, :]
plt.xlabel(r"$\frac{v}{\epsilon}$")
plt.ylabel(r"$\frac{w}{\epsilon}$")
save_fig("lipkin-w-eig.pdf")

(10,) (10,)


In [6]:
# vs = np.arange(0, 2, 0.02)
# ws = np.arange(0, 2, 0.02)
# eigs = np.zeros((5, len(vs)))

# # let v=w
# v=0
# for i, w in enumerate(ws):
#     eigvals, eigvecs = np.linalg.eig(H(0,w))
#     eigs[:,i] = sorted(eigvals)

# plot_2D(vs, eigs[0], plot_count=1, title=r"Numerical Eigenvalues for $J=2$ with v=w", x_title=r'$v$', 
#         label = [rf'$E_{i}$' for i in range(5)], y_title='Energies for v=1')

In [7]:
# plot_2D(vs, eigs, plot_count=5, title=r"Numerical Eigenvalues for $J=2$ with v=w", x_title=r'$v$', 
#         label = [rf'$E_{i}$' for i in range(5)], y_title='Energies for v=w', filename='lipkin-eigs-4-w.pdf')

In [8]:
# ax.plot_surface(vs, ws, eigs[0, :, :])  # Plotting the first eigenvalue (you can change the index to plot different eigenvalues)

# Create a heatmap using Seaborn


As we can see, every eigenvalue has a degenerate value. However $E_3$ and $E_{14}$ are energy eigenvalues that we do not get from the 5x5 matrix. 

In [9]:
def my_H(v, w):
    
    x_coeff = -v/2 + w
    y_coeff = v/2 + w
    return PauliSumOp.from_list([('IIII', 2*w), 
                                ('ZIII', z_coeff), ('IZII', z_coeff), ('IIZI', z_coeff), ('IIIZ', z_coeff), # all 1 qubit combinations
                                ('XXII', x_coeff), ('XIXI', x_coeff), ('XIIX', x_coeff), ('IXXI', x_coeff), ('IXIX', x_coeff), ('IIXX', x_coeff), # all 2 qubit combinations
                                ('YYII', y_coeff), ('YIYI', y_coeff), ('YIIY', y_coeff), ('IYYI', y_coeff), ('IYIY', y_coeff), ('IIYY', y_coeff)])

# my_H(0)

In [10]:
def prep_vqe(n,reps):
    '''input:
        n: int, number of qubits'''
    adam = ADAM(maxiter = 10000)
    slsqp = SLSQP(maxiter = 5000)
    cobyla = COBYLA(maxiter = 500)
    # backend = Aer.get_backend('qasm_simulator')
    estimator = Estimator()
    ansatz = TwoLocal(n, ['rx', 'ry'], 'cx', reps=reps, entanglement='full')
    vqe = VQE(ansatz = ansatz,
              optimizer=slsqp, 
              initial_point=np.random.uniform(low=-np.pi, high=np.pi, size=2 * n * (reps+1)), 
              estimator=estimator)

    return vqe

In [11]:
n_qubits=4

vs = np.arange(0, 1, 0.1) # rmb to divide when used
ws = np.arange(0, 1, 0.1)
n_points = len(vs)
vqe = prep_vqe(n_qubits, reps=1)
energies = np.zeros((len(vs), len(vs))) # containers for the energies

for i, v in enumerate(vs):
    for j, w in enumerate(ws):
        # print(ham)
        result = vqe.compute_minimum_eigenvalue(my_H(v,w))
        energies[i, j] = result.eigenvalue.real
        print(f'v = {v}, w = {w}, energy = {result.eigenvalue.real}')




v = 0.0, w = 0.0, energy = -1.9999999506667598
v = 0.0, w = 0.1, energy = -1.7999995950412353
v = 0.0, w = 0.2, energy = -1.5999989949794478
v = 0.0, w = 0.30000000000000004, energy = -0.9999995326817644
v = 0.0, w = 0.4, energy = -0.9999998274754055
v = 0.0, w = 0.5, energy = -0.9999998646248296
v = 0.0, w = 0.6000000000000001, energy = -1.1999997967430203
v = 0.0, w = 0.7000000000000001, energy = -1.3999998107706475
v = 0.0, w = 0.8, energy = -1.5999946317078604
v = 0.0, w = 0.9, energy = -1.7999986672281632
v = 0.1, w = 0.0, energy = -2.015530693163918
v = 0.1, w = 0.1, energy = -1.8137909355651434
v = 0.1, w = 0.2, energy = -1.004986984145781
v = 0.1, w = 0.30000000000000004, energy = -1.004978467582423
v = 0.1, w = 0.4, energy = -1.00497215803783
v = 0.1, w = 0.5, energy = -1.0049860405955966
v = 0.1, w = 0.6000000000000001, energy = -1.1999996337540708
v = 0.1, w = 0.7000000000000001, energy = -1.3999963098685795
v = 0.1, w = 0.8, energy = -1.5999987592875096
v = 0.1, w = 0.9, en

In [30]:
t = np.arange(0,10)
t = t/10
sns.heatmap(energies.T, cmap='viridis', xticklabels=t, yticklabels=t)
plt.title("VQE ground state energy eigenvalues with qiskit")
plt.xlabel(r"$\frac{v}{\epsilon}$")
plt.ylabel(r"$\frac{w}{\epsilon}$")
save_fig("lipkin-4-w.pdf")

In [33]:
def calculate_relative_error(true_value, predicted_value):
    # Calculate the absolute difference between true and predicted values
    abs_diff = abs(true_value - predicted_value)

    # Calculate the relative error
    relative_error = abs_diff / abs(true_value)

    return relative_error
t = np.arange(0,10)
t = t/10
rel_error = calculate_relative_error(eigs[0], energies)
sns.heatmap(rel_error.T, cmap='viridis', xticklabels=t, yticklabels=t)
plt.title("Relative Error for VQE")
plt.xlabel(r"$\frac{v}{\epsilon}$")
plt.ylabel(r"$\frac{w}{\epsilon}$")
save_fig("rel-error.pdf")